In [1]:
import os

import pandas as pd
import deepchem as dc

import pandas as pd

from tqdm import tqdm
import json
import time
import csv
import os
import random

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.linear_model import LinearRegression
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from scipy.spatial.distance import cosine
from scipy.stats import pearsonr
import statsmodels.api as sm


No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
/Users/lyu/anaconda3/envs/transKnow/lib/python3.10/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [89]:
chem_dis = pd.read_csv("./CTD/CTD_chemicals_diseases.csv", comment='#', low_memory=False)
chem_dis["DiseaseID"] = chem_dis["DiseaseID"].str.extract(r'(D\d+)')
chem_dis.head()

,ChemicalName,ChemicalID,CasRN,DiseaseName,DiseaseID,DirectEvidence,InferenceGeneSymbol,InferenceScore,OmimIDs,PubMedIDs
0,06-Paris-LA-66 protocol,C046983,NaN,Precursor Cell Lymphoblastic Leukemia-Lymphoma,D054198,therapeutic,NaN,NaN,NaN,4519131
1,10074-G5,C534883,NaN,Adenocarcinoma,D000230,NaN,MYC,4.07,NaN,26432044
2,10074-G5,C534883,NaN,Adenocarcinoma of Lung,D000077192,NaN,MYC,4.30,NaN,26656844|27602772
3,10074-G5,C534883,NaN,Alopecia,D000505,NaN,AR,4.49,NaN,15902657
4,10074-G5,C534883,NaN,Androgen-Insensitivity Syndrome,D013734,NaN,AR,6.86,300068|312300,1303262|8281139


In [90]:
with open("./PubMed/pubmed_CTD_0.json", 'r', encoding='utf') as f:
    jsons = json.load(f)

json_data = [x for x in jsons if x['abstract_text'] and x['year']]

with open("./PubMed/pubmed_CTD_1.json", 'r', encoding='utf') as f:
    jsons = json.load(f)

json_data += [x for x in jsons if x['abstract_text'] and x['year']]

with open("./PubMed/pubmed_CTD_2.json", 'r', encoding='utf') as f:
    jsons = json.load(f)

json_data += [x for x in jsons if x['abstract_text'] and x['year']]

pmids = []
years = []
abstract_texts = []

for js in json_data:
    pmids.append(str(js['pmid']))
    years.append(js['year'])
    abstract_texts.append(js['abstract_text'])

pmid_df = pd.DataFrame({
    'pmid': pmids,
    'year': years,
    'abstract_text': abstract_texts
})

pmid_df

,pmid,year,abstract_text
0,706,1976,Thirteen out of 18 young out-patients with sim...
1,1803,1976,Mice were given a drug per os and 2 h later we...
2,1959,1976,"In a double-blind trial lasting 2 weeks, a new..."
3,2004,1976,Eight patients had cardiac manifestations that...
4,2127,1976,Beta-Adrenergic stimulation with isoproterenol...
...,...,...,...
9284,3418400,1988,Two episodes of massive bleeding from a suture...
9285,3419247,1988,The anti-amnestic action of nicergoline was st...
9286,3420160,1988,Protective influence of verapamil upon ouabain...
9287,3420309,1988,An unusual case of acute muscular atrophy in a...


In [91]:
Chem_id = pd.read_csv("./Identifier/Chem_id.csv")
chem_dis = chem_dis[chem_dis['ChemicalName'].isin(Chem_id['iupac'])]
len(chem_dis)

2220872

In [92]:
pubMed_ids = []
for s in chem_dis['PubMedIDs']:
    t = str(s).split('|')
    pubMed_ids += t
pubMed_ids = list(set(pubMed_ids))
len(pubMed_ids)

39544

In [93]:
k_l = [x for x in pmid_df['pmid'] if x in pubMed_ids]
len(k_l)

3613

In [94]:
chems = []
diseases = []
pmids = []
abstracts = []

for i, row in chem_dis.iterrows():
    for t in str(row['PubMedIDs']).split('|'):
        if t in k_l:
            chems.append(row['ChemicalName'])
            diseases.append(row['DiseaseName'])
            pmids.append(t)
            abstracts.append(list(pmid_df[pmid_df['pmid'] == t]['abstract_text'])[0])

df_relation = pd.DataFrame({
    'Chemical': chems,
    'Disease': diseases,
    'Pmid': pmids,
    'Text': abstracts
})

df_relation.to_csv("./Dual_sp_core.csv", index=False, sep='$')

In [98]:
pmid_e_df = pmid_df[pmid_df['pmid'].isin(k_l)]
pmid_df.to_csv("./Pmid_e.csv", index=False)

In [99]:
len(pmid_df)

9289

In [100]:
len(df_relation)

61954